In [11]:
#importing the required libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import dask
import dask.dataframe as dd
from scipy.sparse import csr_matrix
import numpy as np
import pickle
from tqdm import tqdm 
import matplotlib.pyplot as plt
import seaborn as sn
from collections import Counter
sn.set()
#from seaborn import heatmap
#import geopandas as gpd
#import geoplot as gplty
#import tqdm as tqdm
#import random
import cmd

In [13]:
#Location of train and test files
train_loc = '../data/raw/train_ver2.csv'
test_loc = '../data/raw/test_ver2.csv'

In [6]:
train_loc

'../data/raw/train_ver2.csv'

In [7]:
test_loc

'../data/raw/test_ver2.csv'

In [ ]:
#We are loading the data as a dask dataframe
data = dd.read_csv(train_loc, dtype={'age': object,
                        'antiguedad': object,
                        'cod_prov': float,
                        'ind_actividad_cliente': float,
                        'ind_nom_pens_ult1': float,
                        'ind_nomina_ult1': float,
                        'ind_nuevo': float,
                        'indrel': float,
                        'tipodom': float,
                        'ult_fec_cli_1t': object,
                        'conyuemp': object,
                        'indrel_1mes': object })

print(data.head())
print('-'*100)
print('Columns in the train data are \n')
columns = list(data.columns)
cmd.Cmd().columnize(columns, displaywidth=80)
print('-'*100)
customers = data.ncodpers.unique()
print('Number of data points in train:', len(data))
print('Number of Unique Customer in train set:', len(customers.compute()))

   fecha_dato  ncodpers ind_empleado pais_residencia sexo  age  fecha_alta  \
0  2015-01-28   1375586            N              ES    H   35  2015-01-12   
1  2015-01-28   1050611            N              ES    V   23  2012-08-10   
2  2015-01-28   1050612            N              ES    V   23  2012-08-10   
3  2015-01-28   1050613            N              ES    H   22  2012-08-10   
4  2015-01-28   1050614            N              ES    V   23  2012-08-10   

   ind_nuevo antiguedad  indrel  ... ind_hip_fin_ult1 ind_plan_fin_ult1  \
0        0.0          6     1.0  ...                0                 0   
1        0.0         35     1.0  ...                0                 0   
2        0.0         35     1.0  ...                0                 0   
3        0.0         35     1.0  ...                0                 0   
4        0.0         35     1.0  ...                0                 0   

  ind_pres_fin_ult1 ind_reca_fin_ult1 ind_tjcr_fin_ult1 ind_valo_fin_ult1  \
0  

In [ ]:
data.compute()

In [ ]:
#Lets change the feature names from spainish to english based on the meaning
cols = ['fetch_date', 'cust_code', 'emp_index', 'country', 'sex', 'age', 'cust_date', 'new_cust', 'cust_seniority',
'indrel', 'last_date_as_primary', 'cust_type', 'cust_rel', 'residence_index', 'foreigner_index', 'spouse_index',
'joining_channel', 'deceased', 'address_type', 'prov_code','prov_name', 'activity_index', 'income', 'segmentation', 
'savings_account', 'guarentees', 'current_account', 'derivative_account', 'payroll_account', 'junior_account', 'mas_account',
'perticular_account', 'perticular_plus', 'st_deposit', 'mt_deposits', 'lt_deposits', 'e_account', 'funds', 'mortgage',
 'pension', 'loan', 'tax', 'credit_card', 'securities', 'home_account', 'payroll', 'pension2', 'direct_debit'
]
data.columns = cols

print('New column names \n')
cmd.Cmd().columnize(cols, displaywidth=80)

In [ ]:
data.head()

In [ ]:
data.isnull().sum().compute()

In [ ]:
def preprocess_missing_values(data, null_columns, target_columns):

  for column in null_columns:
    if column not in target_columns:
      print('preprocessing', column)
      custs = data.cust_code[data[column].isna()].unique().compute()
      print('Number of customers whose {} data is missing is {}'.format(column, custs.shape[0]))
      available_custs = data.cust_code[(~data[column].isna()) & (data.cust_code.isin(custs))].unique().compute()
      print('Number of customers whose data is present elsewhere is {}'.format(available_custs.shape[0]))

      #Creating a dummy data set as we can not assign values to dask dataframes
      dummy = data[data.cust_code.isin(available_custs)].compute()
      data = data[~data.cust_code.isin(available_custs)]

      available_values = dummy[(dummy.cust_code.isin(available_custs))&(~dummy[column].isna())].groupby(by='cust_code')[column].first()
      available_dict = dict(zip(available_values.index, available_values.values))
      dummy[column][(dummy.cust_code.isin(available_custs))&(dummy[column].isna())] = dummy['cust_code'][(dummy.cust_code.isin(available_custs))&(dummy[column].isna())].map(available_dict)
      print('processed', column)
      data = dd.concat([data, dd.from_pandas(dummy, npartitions = 20)])
      print('-'*100)
    else:
      print('preprocessing', column)
      print('filling na values with 0')

      dummy = data[data[column].isna()].compute()
      dummy[column] = 0
      data = data[~data[column].isna()]
      data = dd.concat([data, dd.from_pandas(dummy, npartitions = 20)])
      print('processed', column)
      print('-'*100)
  
  return data

In [ ]:
#ts
target_columns = ['savings_account', 'guarentees', 'current_account', 'derivative_account', 'payroll_account', 'junior_account', 'mas_account',
'perticular_account', 'perticular_plus', 'st_deposit', 'mt_deposits', 'lt_deposits', 'e_account', 'funds', 'mortgage',
 'pension', 'loan', 'tax', 'credit_card', 'securities', 'home_account', 'payroll', 'pension2', 'direct_debit']

user_features = ['fetch_date', 'cust_code', 'emp_index', 'country', 'sex', 'age', 'cust_date', 'new_cust', 'cust_seniority',
'indrel', 'last_date_as_primary', 'cust_type', 'cust_rel', 'residence_index', 'foreigner_index', 'spouse_index',
'joining_channel', 'deceased', 'address_type', 'prov_code','prov_name', 'activity_index', 'income', 'segmentation']

In [ ]:
target_columns

In [ ]:
user_features

In [ ]:
null_columns = data.isna().sum()[data.isna().sum()>0].index.compute()
data = preprocess_missing_values(data, null_columns, target_columns)

If a product is bought at once it is carried and mentioned in all the subsequent months, Lets change this format, Lets create a new dataframe where we keep it mentioned in only in the month its bought.

We will remove jan 2015 data as it carries all the purchases from the past

In [ ]:
#creating a new dataframe for a previous date
dummy = pd.DataFrame(
    {
        'cust_code': data.cust_code.unique().compute(),
        'fetch_date': '2014-12-28'
    }
)

new_purchases = pd.concat([data[['cust_code', 'fetch_date']+target_columns].compute(), dummy])

new_purchases = new_purchases.fillna(0)
new_purchases[target_columns] = new_purchases[target_columns].astype('uint8')
new_purchases = new_purchases.sort_values(['cust_code', 'fetch_date'])

vals = np.array(new_purchases[target_columns].values, dtype='int8')
vals[1:] = vals[1:] - vals[:-1]

new_purchases[target_columns] = vals
#Removing the data of '2014-12-28' and '2015-01-28'
new_purchases = new_purchases[~new_purchases.fetch_date.isin(['2014-12-28','2015-01-28'])]

#Some of the products were discontinued so purchase value there becomes less than 0,
#as we are only interested in purchases we can remove them
for col in target_columns:
  new_purchases[col][new_purchases[col] < 0] = 0

#drop all the rows where no new purchase is made
new_purchases = new_purchases[(new_purchases[target_columns].sum(axis=1) > 0)]

new_purchases = data[user_features].merge(dd.from_pandas(new_purchases, npartitions=3), on = ['fetch_date', 'cust_code'], how='right')

Some of the features are of the users which do not change with each purchase, We can make a dataset of those features

In [ ]:
user_data = new_purchases.drop_duplicates(subset=['cust_code'], keep='last')[['cust_code', 'emp_index', 'country', 'sex', 'age', 'cust_date', 'new_cust', 'cust_seniority',
                  'indrel', 'cust_type', 'cust_rel', 'residence_index', 'foreigner_index', 'spouse_index',
                  'joining_channel', 'deceased', 'address_type', 'prov_code','prov_name', 'activity_index',
                  'income', 'segmentation']]

user_purchases = new_purchases[['cust_code']+target_columns].groupby(by='cust_code').sum()

user_features = user_data.merge(user_purchases, on='cust_code', how='right')

In [ ]:
new_purchases = pd.read_csv('/content/drive/MyDrive/Santander/new_purchases.csv')
user_features = pd.read_csv('/content/drive/MyDrive/Santander/user_features.csv')

In [ ]:
print(new_purchases.info())


In [ ]:
def plot_distribution(series, plt_title, figsize, bins=14):
  plt.figure(figsize = figsize)
  plt.title(plt_title)
  series = np.array(series, dtype=int)
  sn.distplot(series, hist=True, bins=bins, kde=True, kde_kws={'bw':.2})
  plt.axvline(series.mean(),color='midnightblue',label='Mean')    
  plt.axvline(np.median(series),color='blue',label='Median')
  plt.axvline(series.max(),color='indigo',label='Max')
  plt.axvline(series.min(),color='crimson',label='Min')
  plt.axvline(np.quantile(series, 0.25),color='red',label='First quartile - 25%')
  plt.axvline(np.quantile(series, 0.75),color='orangered',label='Third quartile - 75%')
  plt.legend()
  plt.show()

def stripplot(data, title, figsize):
  plt.figure(figsize=figsize)
  plt.title(title)
  y = data.columns[0]
  x = data.columns[1]
  sn.stripplot(x=x, y=y, data=data, jitter=False, dodge=True)
  sn.boxplot(x=x, y=y, data=data)
  plt.xticks(rotation = 90)
  plt.show()

def print_count(series, title, show=True):
  counts = Counter(series)
  countlist = []
  for c in counts:
    countlist.append(str(c) + ':' +str(counts[c]))
  if show:
    print('\n'+title)
    cmd.Cmd().columnize(countlist, displaywidth=80)

  return counts

def piecountplot(series, title):
  plt.title(title)
  series.value_counts().plot(kind='pie', autopct='%1.2f%%')
  plt.legend()

def piepurchaseplot(dictionary, title):
  plt.title(title)
  plt.pie(dictionary.values(), labels = dictionary.keys(), autopct='%1.2f%%')
  plt.legend()

def countplot(series, title, figsize):
  plt.figure(figsize=figsize)
  plt.title(title)
  sn.countplot(series)
  plt.show()

def barplot(series, title, figsize):
  plt.figure(figsize=figsize)
  plt.title(title)
  plt.bar(series.index, series.values, color="royalblue")
  plt.show()

def allplots(data, title, kind='countplot', figsize=(10,2.5)):
  if kind == 'countplot':
    countplot(data, title, figsize)
  elif kind == 'piecountplot':
    piecountplot(data, title)
  elif kind == 'dist_plot':
    plot_distribution(data, title, figsize)
  elif kind == 'barplot':
    barplot(data, title, figsize)
  elif kind == 'piepurchaseplot':
    piepurchaseplot(data, title)
  elif kind == 'stripplot':
    stripplot(data, title, figsize)

Analyzing fetch_date

In [ ]:
#plotting number of purchases in each month
dummy = new_purchases[['fetch_date']+target_columns].groupby('fetch_date').sum()
dummy.plot(kind='bar',stacked=True, colormap='rainbow',figsize=(20,5))
plt.legend(loc='center left', title='Legend Title', bbox_to_anchor=(1, .4))
plt.title('Number of purchases in each month')
plt.show()
print('-'*150)
#Plotting the distribution of products in each month
dummy = new_purchases[['fetch_date']+target_columns].groupby('fetch_date').sum()
dummy = dummy/dummy.sum()
dummy = dummy.T
dummy.plot(kind='bar',stacked=True, colormap='rainbow',figsize=(20,5))
plt.legend(loc='center left', title='Legend Title', bbox_to_anchor=(1, .4))
plt.title('Distribution of products in each month')
plt.show()
